In [33]:
!pip install Feature-engine
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from feature_engine.creation import CyclicalFeatures
import scipy.stats as stats
from google.colab import drive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
drive.mount('/content/drive/', force_remount=True)
fileName = "/content/drive/My Drive/Dataset.xlsx"
df_Labels = pd.read_excel(fileName, sheet_name="Plant_1_Data")
df_Labels = df_Labels.drop(['PLANT_ID'], axis=1)
df_Labels["DATE_TIME"] = pd.to_datetime(df_Labels.DATE_TIME)
df_Features = pd.read_excel(fileName, sheet_name="Plant_1_Sensor_Data")
df_Features = df_Features.drop(['PLANT_ID','SOURCE_KEY'], axis=1)
df_Features['DATE_TIME'] = pd.to_datetime(df_Features.DATE_TIME)
print(df_Features.head())

Mounted at /content/drive/
            DATE_TIME  AMBIENT_TEMPERATURE  MODULE_TEMPERATURE  IRRADIATION
0 2020-05-15 00:00:00            25.184316           22.857507          0.0
1 2020-05-15 00:15:00            25.084589           22.761668          0.0
2 2020-05-15 00:30:00            24.935753           22.592306          0.0
3 2020-05-15 00:45:00            24.846130           22.360852          0.0
4 2020-05-15 01:00:00            24.621525           22.165423          0.0


In [35]:
df_Learn = pd.merge(df_Features, df_Labels, how='inner', left_on = ['DATE_TIME'], right_on = ['DATE_TIME'])
df_Learn

,DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,2020-05-15 00:00:00,25.184316,22.857507,0.0,1BY6WEcLGh8j5v7,0.0,0.0,0.000,6259559.0
1,2020-05-15 00:00:00,25.184316,22.857507,0.0,1IF53ai7Xc0U56Y,0.0,0.0,0.000,6183645.0
2,2020-05-15 00:00:00,25.184316,22.857507,0.0,3PZuoBAID5Wc2HD,0.0,0.0,0.000,6987759.0
3,2020-05-15 00:00:00,25.184316,22.857507,0.0,7JYdWkrLSPkdwr4,0.0,0.0,0.000,7602960.0
4,2020-05-15 00:00:00,25.184316,22.857507,0.0,McdE0feGgRqW7Ca,0.0,0.0,0.000,7158964.0
...,...,...,...,...,...,...,...,...,...
45675,2020-06-17 23:45:00,21.909288,20.427972,0.0,uHbuxQJl8lW7ozc,0.0,0.0,5967.000,7287002.0
45676,2020-06-17 23:45:00,21.909288,20.427972,0.0,wCURE6d3bPkepu2,0.0,0.0,5147.625,7028601.0
45677,2020-06-17 23:45:00,21.909288,20.427972,0.0,z9Y9gH1T5YWrNuG,0.0,0.0,5819.000,7251204.0
45678,2020-06-17 23:45:00,21.909288,20.427972,0.0,zBIq5rxdHJRwDNY,0.0,0.0,5817.000,6583369.0


In [36]:
'''sourceKeyList = list(df_Learn['SOURCE_KEY'].unique())
sourceKeyDict = {}
for key in sourceKeyList:
  df_Key = df_Learn[df_Learn['SOURCE_KEY'] == key]
  #print(df_Key.describe())
  X = df_Key.drop(['DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD', 'SOURCE_KEY'], axis = 1)
  #print(X.describe())
  y = df_Key['DC_POWER']
  
  #Train Val Test Split: 80%, 10%, 10%
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=1/9, random_state=42)
  #print(X_train.head()) print(y_train)
  sourceKeyDict[key] = list([X_train, y_train, X_val, y_val, X_test, y_test])
len(sourceKeyDict)
'''
X = df_Learn.drop(['DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD'], axis = 1)
X['hour'] = X['DATE_TIME'].dt.hour
X['min'] = X['DATE_TIME'].dt.minute
X = X.drop(['DATE_TIME'], axis = 1)
y = np.log(df_Learn['DC_POWER'] + 1)
display(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cols_to_scale = ['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']
scaler = StandardScaler()
scaler.fit(X_train[cols_to_scale])
X_train[cols_to_scale] = scaler.transform(X_train[cols_to_scale])
X_test[cols_to_scale] = scaler.transform(X_test[cols_to_scale])
display(X_train)


,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,SOURCE_KEY,hour,min
0,25.184316,22.857507,0.0,1BY6WEcLGh8j5v7,0,0
1,25.184316,22.857507,0.0,1IF53ai7Xc0U56Y,0,0
2,25.184316,22.857507,0.0,3PZuoBAID5Wc2HD,0,0
3,25.184316,22.857507,0.0,7JYdWkrLSPkdwr4,0,0
4,25.184316,22.857507,0.0,McdE0feGgRqW7Ca,0,0
...,...,...,...,...,...,...
45675,21.909288,20.427972,0.0,uHbuxQJl8lW7ozc,23,45
45676,21.909288,20.427972,0.0,wCURE6d3bPkepu2,23,45
45677,21.909288,20.427972,0.0,z9Y9gH1T5YWrNuG,23,45
45678,21.909288,20.427972,0.0,zBIq5rxdHJRwDNY,23,45


,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,SOURCE_KEY,hour,min
10398,-1.095301,-0.979870,-0.772065,adLQvlD726eNBSB,3,15
27604,-0.629739,0.270719,0.401938,3PZuoBAID5Wc2HD,8,30
43378,-0.681902,-0.796498,-0.772065,ZnxXDlPa8U1GXgE,21,15
41261,-0.809548,-0.850050,-0.772065,7JYdWkrLSPkdwr4,21,15
42310,-0.139878,0.410836,0.570744,wCURE6d3bPkepu2,9,0
...,...,...,...,...,...,...
11284,1.582100,1.934327,2.177753,sjndEbLyjtCKgGv,13,15
44732,0.753294,0.924885,0.827891,zBIq5rxdHJRwDNY,13,0
38158,0.127217,0.905271,1.142164,3PZuoBAID5Wc2HD,10,0
860,0.768115,1.639194,1.420128,1BY6WEcLGh8j5v7,10,0


In [37]:
display(len(y_train))
display(len(X_train))

36544

36544

In [38]:
cyclical = CyclicalFeatures(variables=['hour', 'min'], drop_original=True)
display(X_train)
cyclical.fit(X_train)
X_train = cyclical.transform(X_train)
X_test = cyclical.transform(X_test)

X_train.shape[1] + 1

,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,SOURCE_KEY,hour,min
10398,-1.095301,-0.979870,-0.772065,adLQvlD726eNBSB,3,15
27604,-0.629739,0.270719,0.401938,3PZuoBAID5Wc2HD,8,30
43378,-0.681902,-0.796498,-0.772065,ZnxXDlPa8U1GXgE,21,15
41261,-0.809548,-0.850050,-0.772065,7JYdWkrLSPkdwr4,21,15
42310,-0.139878,0.410836,0.570744,wCURE6d3bPkepu2,9,0
...,...,...,...,...,...,...
11284,1.582100,1.934327,2.177753,sjndEbLyjtCKgGv,13,15
44732,0.753294,0.924885,0.827891,zBIq5rxdHJRwDNY,13,0
38158,0.127217,0.905271,1.142164,3PZuoBAID5Wc2HD,10,0
860,0.768115,1.639194,1.420128,1BY6WEcLGh8j5v7,10,0


9

Feature Transformations:

In [39]:
###Polynomial Features

from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree = 3)
poly.fit(X_train.drop(['SOURCE_KEY'], axis = 1))
X_train_poly = pd.DataFrame(poly.transform(X_train.drop(['SOURCE_KEY'], axis = 1)))
X_test_poly = pd.DataFrame(poly.transform(X_test.drop(['SOURCE_KEY'], axis = 1)))
display(X_train_poly)

,0,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
0,1.0,-1.095301,-0.979870,-0.772065,0.730836,0.682553,0.866025,-0.5,1.199684,1.073252,...,0.317987,0.403463,-0.232939,0.511915,-0.295554,0.170638,0.649519,-0.375,0.216506,-0.125
1,1.0,-0.629739,0.270719,0.401938,0.816970,-0.576680,-0.866025,-0.5,0.396571,-0.170483,...,-0.191781,-0.288006,-0.166280,-0.432510,-0.249710,-0.144170,-0.649519,-0.375,-0.216506,-0.125
2,1.0,-0.681902,-0.796498,-0.772065,-0.519584,0.854419,0.866025,-0.5,0.464991,0.543134,...,0.623754,0.632227,-0.365016,0.640815,-0.369974,0.213605,0.649519,-0.375,0.216506,-0.125
3,1.0,-0.809548,-0.850050,-0.772065,-0.519584,0.854419,0.866025,-0.5,0.655368,0.688157,...,0.623754,0.632227,-0.365016,0.640815,-0.369974,0.213605,0.649519,-0.375,0.216506,-0.125
4,1.0,-0.139878,0.410836,0.570744,0.631088,-0.775711,0.000000,1.0,0.019566,-0.057467,...,-0.466767,0.000000,0.601728,-0.000000,-0.000000,-0.775711,0.000000,0.000,0.000000,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36539,1.0,1.582100,1.934327,2.177753,-0.398401,-0.917211,0.866025,-0.5,2.503040,3.060299,...,-0.771628,0.728567,-0.420638,-0.687908,0.397164,-0.229303,0.649519,-0.375,0.216506,-0.125
36540,1.0,0.753294,0.924885,0.827891,-0.398401,-0.917211,0.000000,1.0,0.567451,0.696710,...,-0.771628,0.000000,0.841277,-0.000000,-0.000000,-0.917211,0.000000,0.000,0.000000,1.000
36541,1.0,0.127217,0.905271,1.142164,0.398401,-0.917211,0.000000,1.0,0.016184,0.115166,...,-0.771628,0.000000,0.841277,-0.000000,-0.000000,-0.917211,0.000000,0.000,0.000000,1.000
36542,1.0,0.768115,1.639194,1.420128,0.398401,-0.917211,0.000000,1.0,0.590001,1.259090,...,-0.771628,0.000000,0.841277,-0.000000,-0.000000,-0.917211,0.000000,0.000,0.000000,1.000


The following code snippets for Power Transformer, PCA, Factor Analysis and Spline Transformer have been commented for faster execution of the original proposed apporach of RANSAC Regression with Polynomial Features.

To run RANSAC with any of the following 4 feature representations, uncomment code and replace "X_train_linear" and "X_test_linear" with variables from the respective feature representations.

In [40]:
###Power Transformer feature representation

# from sklearn.preprocessing import PowerTransformer

# pt = PowerTransformer()

# pt.fit(X_train.drop(['SOURCE_KEY'], axis = 1))
# X_train_pt = pt.transform(X_train.drop(['SOURCE_KEY'], axis = 1))
# X_test_pt = pt.transform(X_test.drop(['SOURCE_KEY'], axis = 1))


In [41]:
### PCA feature representation - Run after running Polynomial Features for omptimized output

# from sklearn.decomposition import PCA
# pca = PCA(n_components=50) #decomposition.PCA(n_components = 2)
# pca.fit(X_train_linear)
# train_pca = pca.transform(X_train_linear)
# test_pca = pca.transform(X_test_linear)


In [42]:
### Factor Analysis feature representation

# from sklearn.decomposition import FactorAnalysis

# factorAnalysis = FactorAnalysis(n_components=4, random_state=42)

# X_train_fa = factorAnalysis.fit_transform(X_train.drop(['SOURCE_KEY'], axis = 1))
# X_test_fa = factorAnalysis.fit_transform(X_test.drop(['SOURCE_KEY'], axis = 1))

In [43]:
### Spline Feature Representation

# from sklearn.preprocessing import SplineTransformer

# spline = SplineTransformer(degree=3, n_knots=3)
# spline.fit(X_train.drop(['SOURCE_KEY'], axis=1))
# X_train_spline = spline.transform(X_train.drop(['SOURCE_KEY'], axis=1))
# X_test_spline = spline.transform(X_test.drop(['SOURCE_KEY'], axis=1))

Random sample selection for RANSAC

In [44]:
###Random partition train samples:

def rand_part(n, train_data):
  np.random.seed(seed = 42)
  total_indexes = np.arange(train_data)
  np.random.shuffle(total_indexes)
  index_min = total_indexes[:n]
  index_test = total_indexes[n:]
  return index_min, index_test


Linear Regression to be used inside RANSAC

In [45]:
# class my LR

class my_Linear_Reg():
  
  def __init__(self, epochs = 1, lr = 1):
    self.w = None
    self.b = None
    self.epochs = epochs
    self.lr = lr
    
  def model(self, x):
    return (np.dot(self.w, x) + self.b)
    
  def predict(self, X):
    
    Y = []
    for x in X:
      result = self.model(x)
      Y.append(result)
    
    return np.array(Y)
  
  def fit(self, X, Y, epochs = 1, lr = 1):
    
    self.w = np.ones(X.shape[1])
    self.b = 0.01
    epochs = self.epochs
    lr = self.lr

    for i in range(epochs):
      for x, y in zip(X, Y):
        y_pred = self.model(x)
        self.w = self.w + lr * (y - y_pred) * x
        self.b = self.b + lr * (y - y_pred)

    return self




RANSAC Regression method

In [46]:
def my_RANSAC(train_data_X, train_data_y, selected_model, min_samples, max_itr, inlier_threshold, stop_inliers):
  itr = 0
  best_fit_model = None
  best_error = np.inf

  while itr < max_itr:
    min_samples_indexes, test_indexes = rand_part(min_samples, train_data_X.shape[0])
    min_samples_X = train_data_X[min_samples_indexes]
    min_samples_y = train_data_y[min_samples_indexes]
    test_samples_X = train_data_X[test_indexes]
    test_samples_y = train_data_y[test_indexes]

    possible_model = selected_model.fit(min_samples_X, min_samples_y)

    y_pred = possible_model.predict(test_samples_X)
    error = np.abs(test_samples_y - y_pred)


    potential_inliers_index = test_indexes[error < inlier_threshold]
    potential_inliers_X = train_data_X[potential_inliers_index]
    potential_inliers_y = train_data_y[potential_inliers_index]

    if len(potential_inliers_X) > stop_inliers:
      better_data_X = np.concatenate((min_samples_X, potential_inliers_X))
      better_data_y = np.concatenate((min_samples_y, potential_inliers_y))
      better_model = selected_model.fit(better_data_X, better_data_y)
      better_model_y_pred = better_model.predict(better_data_X)
      better_model_err = mean_absolute_error(better_data_y, better_model_y_pred)

      if better_model_err < best_error:
        best_fit_model = better_model
        best_error = better_model_err

    itr += 1

  if best_fit_model is None:
    raise ValueError("Could not find a suiable model")
  else:
    return best_fit_model







Call RANSAC with hyper-parameters

In [47]:
#ransac_estimator = LinearRegression()

#Initialize Linear Regression model to be passed to RANSAC as an estimator
ransac_estimator = my_Linear_Reg(epochs = 100, lr = 0.0001) 
y_train_MAD = round(((pd.Series(y_train)).mad()), 4)      # Median Absolute Deviation for inlier threshold
display(y_train_MAD)

#Call RANSAC Regressor 
regr_ransac = my_RANSAC(X_train_poly.values, y_train.values, ransac_estimator, 10000, 5, y_train_MAD, 60)

#Test RANSAC model by running prediction on test data
y_pred_ransac = regr_ransac.predict((X_test_poly).values)

mae_ransac = mean_absolute_error(y_test, y_pred_ransac)
display(mae_ransac)

root_mean_squared_error = mean_squared_error(y_true = y_test, y_pred = y_pred_ransac)
display(root_mean_squared_error)


4.1223

0.5005843493962454

0.6026627828192922

In [48]:
#display(X_test)
#vals = pd.DataFrame([[pd.to_datetime('2020-05-15 12:30:00'), '32.14768473',	'52.35325513',	'0.6492476293', 'HmiyD2TTLFNqkNe']], columns=('DATE_TIME', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION', 'SOURCE_KEY'))
vals = pd.DataFrame([[pd.to_datetime('2020-05-15 12:30:00'), '32.14768473',	'52.35325513',	'0.6492476293', '1BY6WEcLGh8j5v7']], columns=('DATE_TIME', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION', 'SOURCE_KEY'))
#display(vals)
vals['hour'] = vals['DATE_TIME'].dt.hour
vals['min'] = vals['DATE_TIME'].dt.minute
vals = vals.drop(['DATE_TIME'], axis = 1)
display(vals)
vals[cols_to_scale] = scaler.transform(vals[cols_to_scale])
#display(vals)
vals = cyclical.transform(vals)
vals = pd.DataFrame(poly.transform(vals.drop(['SOURCE_KEY'], axis = 1)))
predicted_value = regr_ransac.predict(vals.values)
if predicted_value < 0:
  predicted_value = 0.0
display("DC_POWER Prediction for sample: " + str(np.exp(predicted_value) - 1))

,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,SOURCE_KEY,hour,min
0,32.14768473,52.35325513,0.6492476293,1BY6WEcLGh8j5v7,12,30


'DC_POWER Prediction for sample: [9869.23546651]'

In [49]:
display("Size of samples: " + str(len(y_test)))
#Variance check for two tailed test
display("Variance of Predicted Samples: " + str(np.var(y_pred_ransac)), "Variance of Ground Truth: " + str(np.var(y_test)))
display("Standard Deviation of the Sample: " + str(np.sqrt(np.var(y_pred_ransac))))
#Ratio
display("Ratio to check for nearly equal Variance: " + str(np.var(y_pred_ransac)/np.var(y_test)))

'Size of samples: 9136'

'Variance of Predicted Samples: 17.2396340378191'

'Variance of Ground Truth: 17.764036084557613'

'Standard Deviation of the Sample: 4.152063828726517'

'Ratio to check for nearly equal Variance: 0.9704795664542486'

In [50]:
#T-Test
#Equal Variance, hence equal_var = 'True'
tstat, t_pval = stats.ttest_ind(a=y_pred_ransac, b=y_test, equal_var=True)

display("T-Statistic: " + str(tstat.round(3)), "PValue: " + str(t_pval.round(3)))

'T-Statistic: -0.018'

'PValue: 0.986'